#  CNN (konvolucione neuralne mreže)

**"A class of deep neural networks most commonly applied to analyzing visual imagery"**



## Pregled
* CNN
* Slojevi
* Novi pogled na protok podataka
* Arhitektura
* Conv
* Pool
* Arhitektura - ponovo
* Česti problemi

## Konvolucione neuralne mreže (ConvNet, CNN)
* Tip NN specijalno namenjen slučajevima kada su ulazni podaci slike
  * Opštije: kada je umesto feature vektora ulaz predstavljen matricom pri čemu postoje pravilnosti u obe dimenzije
* **Kao i MLP**: feedforward mreža, sastoji se od neurona sa aktivacionim funkcijama, može imati raznovrsnu arhitekturu, na isti način se vrši forward propagation, iste su funkcije troška, na isti način se vrši trening (backpropagation od troška ka neuronima)...
* **Ono što se razlikuje od MLP**: novi tipovi slojeva specijalno konstruisani za rad sa slikama, novi pogled na protok podataka, standardni šabloni po pitanju arhitekture

## Slojevi
* U slučaju MLP imali smo samo jedan tip sloja: **potpuno povezan** (FullyConnected, FC, Dense)
  * Ovaj sloj je definisan trojkom  $(W, b, f)$ - matrica težina, bias vektor, funkcija aktivacije
  * Ulaz (vektor) množi sa $W$, dodaje $b$ i zatim primenjuje $f$ kako bi se dobio novi vektor
* U slučaju CNN uvodimo dva nova, malo drukčija, sloja: **konvoluciju** (Conv) i **pooling** (Pool) - biće objašnjeni vrlo uskoro
* Takođe, aktivacione funkcije možemo posmatrati kao slojeve za sebe (a i ne moramo)

## Novi pogled na protok podataka
* CNN svaki međurezultat posmatraju (umesto kao feature vektor) kao 3D kvadar $(W,H,C)$ (širina, visina, broj kanala tj. dubina)
* Ulazni podaci su slika tj. kvadar tj. 3D tenzor $(W, H, 3)$ u slučaju RGB ili $(W, H, 1)$ u slučaju BW slika
  
## Arhitektura
* Jedna CNN će se sastojati od niza Conv i Pool slojeva koji transformišu kvadar (tj. sliku) u novi kvadar (tj. novu sliku, iako dobijene slike brzo prestaju da imaju vizuelno smisla)
* http://cs231n.github.io/assets/nn1/neural_net2.jpeg
* Na samom kraju ćemo uglavnom imati jedan ili više FC slojeva koji, kao i inače, rade nad vektorskom reprezentacijom
* Dakle: Conv i Pool transformišu sliku tj. skup feature-a a finalni FC slojevi (tj. finalni MLP) koristi te feature-e umesto originalnih kao vektor nad kojim vrši predikciju
* http://cs231n.github.io/assets/cnn/convnet.jpeg
* https://cdn-images-1.medium.com/max/1600/1*uAeANQIOQPqWZnnuH-VEyw.jpeg

## Konvolucija
* Motivacija: Primena FC sloja na RGB sliku 200x200 zahteva 120000 težina po neuronu. Ovaj broj je veliki jer svaki neuron "posmatra" svaki Treba nam sloj koji bolje koristi prostorne pravilnosti u slici a ima manje parametara.
* Ideja: Svaki neuron posmatra samo deo ulaza umesto ceo ulaz (sada je jasno zašto se FC zove FC tj. Dense)
* Podsetnik: ulaz je 3D tenzor $(W, H, C)$
* Konvolucioni sloj se sastoji od niza kernela (filtera)
* Jedan kernel je 3D tenzor $(K, K, C)$
* Svaki kernel proizvodi jedan kanal izlazne slike ($N_K$ kernela daje izlazni 3D tenzor dimenzija $(W', H', N_K)$, gde se $W'$ i $H'$ mogu lako izračunati)
* Sada se fokusirajmo na jedan kernel $(K, K, C)$ i kako on vrši transformaciju $(W,H,C) \to (W', H')$
* Ovde dolazimo do operacije **konvolucije**: prevlačimo kernel preko ulazne slike po širini i dužini (kernel je uvek iste dubine tj. broja kanala kao ulaz) i za svaku poziciju računamo skalarni proizvod vrednosti u ulazu i u kernelu i to upisujemo kao jednu od vrednosti na izlazu
* Posmatrajmo za početak slučaj C=1
* https://cdn-images-1.medium.com/max/1200/1*nYf_cUIHFEWU1JXGwnz-Ig.gif
* https://cdn-images-1.medium.com/max/1200/1*1VJDP6qDY9-ExTuQVEOlVg.gif
* https://cdn-images-1.medium.com/max/1200/1*uoWYsCV5vBU8SHFPAPao-w.gif
* Animacija na http://cs231n.github.io/convolutional-networks/
* U kontekstu neurona jedan kernel se može posmatrati kao $W' * H'$ neurona pri čemu svaki ima svoje **receptivno polje** (dakle nije "dense") i svi dele težine (ako su bitne horizontalne ivice, bitne su svuda => iste težine) - ovo dovodi do bolje efikasnosti Conv slojeva tj. manjeg broja težina (tačan broj težina se lako računa)
* Po dubini, svi neuroni imaju isto receptivno polje
* http://cs231n.github.io/assets/cnn/depthcol.jpeg
* Parametri: broj kernela, veličina kernela, stride, padding (isti za jedan Conv sloj)
* Rezultat: jednostavni feature-i na nižim, i kompleksniji na višim slojevima
* http://cs231n.github.io/assets/cnn/weights.jpeg

## Pooling
- Za razliku od Conv, Pool je dosta jednostavniji sloj **bez learnable parametara** 
- Ima dva fiksna parametra, dimenziju i stride, i služi da smanji $W$ i $H$ (zadržava broj kanala istim)
- Svaki pool sloj deli ulaznu sliku (po širini i dužini) na kvadrate fiksne veličine (uglavnom 2 ili 3) i "sumira" ih nekom funkcijom (max, min, avg)
- http://cs231n.github.io/assets/cnn/pool.jpeg
- Trivia: FCN (Pool se može zameniti Conv-om a FC predstaviti kao Conv i dobijamo mrežu koja ima samo konvolucije)

## Arhitektura - ponovo
- Najčešće:

```
 [[CONV -> RELU]*N -> POOL?]*M -> [FC -> RELU]\*K -> FC
```
- ILSVRC: takmičenje u klasifikaciji slika na ImageNet skupu (1000 kategorija, 1.2M slika)
- Često se koristi kao merilo "najbolje" konvolucione mreže
- Skorašnji pobednici: LeNet, AlexNet, GoogLeNet, VGGNet, ResNet
- Transfer learning: mrežu treniranu za jedan zadatak primenjujemo na drugi, tako možemo konvolucioni deo  ResNet-a pretreniranog na ImageNet (koji sami ne bismo nikad uspeli da istreniramo) da primenimo na bilo koji problem kao feature extraction korak i onda samo dodamo MLP na kraj

## Česti problemi
- https://skymind.ai/wiki/convolutional-network

## Resursi
- http://cs231n.github.io/convolutional-networks/ (odličan detaljan uvod u CNN, intuicija)
- https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53 (kraći uvod, dobre animacije)
- https://skymind.ai/wiki/convolutional-network (pregled problema)

## Implementacija CNN u Keras
* U TF bi takođe bilo relativno jednostavno (`tf.layers.conv2d` i `tf.layers.max_pooling2d`) ali sada nema razloga da ne koristimo Keras (fokus na višem nivou)
* Skup CIFAR-10 za klasifikaciju slika
* 60000 32x32 RGB slika u 10 klasa (svaka klasa ima jednako slika), 5/6 toga je trening test
* Klase: https://www.cs.toronto.edu/~kriz/cifar.html
* "Lakša verzija imagenet"

In [0]:
'''
#Train a simple deep CNN on the CIFAR10 small images dataset.
It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
#save_dir = os.path.join(os.getcwd(), 'saved_models')
#model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
# ONEHOT
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# Opet objasni dropout i batchnorm i DODAJ GORE!!!!!!!!!!!!!!!!!

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
#if not os.path.isdir(save_dir):
#    os.makedirs(save_dir)
#model_path = os.path.join(save_dir, model_name)
#model.save(model_path)
#print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using TensorFlow backend.


 65396736/170498071 [==========>...................] - ETA: 2:32

KeyboardInterrupt: ignored